# SCGNN Test - Kaza - PCA

In [1]:
import pandas as pd
import sys
from pathlib import Path
sys.path.append('../..')
from src import config
from src.utils.dataset import TrafficDataset
from src.utils.model import TrafficPredictor
from torch.utils.data import DataLoader
import torch

In [2]:
DATASET_PATH = config.DATA_KAZA
jsonl_file = DATASET_PATH / "data.jsonl"
if jsonl_file.exists():
    df = pd.read_json(jsonl_file, lines=True)
    if "traffic_sequence" in df.columns:
        df = pd.DataFrame(df["traffic_sequence"].tolist())
else:
    df = pd.read_parquet(DATASET_PATH / "data.parquet")
df = df.fillna(0)
X_STEP, Y_STEP = 2, 1
data_array = df.values

In [3]:
PCA_COMPONENTS = 5
dataset = TrafficDataset(data_array, X_STEP, Y_STEP, pca_components=PCA_COMPONENTS)
sensors = dataset.sensors
edges = []
for i in range(sensors - 1):
    edges.append([i, i + 1])
    edges.append([i + 1, i])
edge_index = torch.tensor(edges, dtype=torch.long).t()
predictor = TrafficPredictor('scgnn', sensors, X_STEP, Y_STEP, edge_index=edge_index, pca_model=dataset.pca if True else None)
predictor.load_model(config.PROJECT_ROOT.joinpath('models/scgnn_kaza_pca.pt'))
loader = DataLoader(dataset, batch_size=10, num_workers=8)
preds = predictor.inference(loader)
display(pd.DataFrame(preds.squeeze()).head().iloc[:, :10])


,0,1,2,3,4
0,-0.044980,-0.044980,-0.044980,-0.016781,-0.010444
1,-0.044980,-0.044980,12.654749,-0.044980,-0.044980
2,-0.044980,-0.044980,-0.044980,-0.044980,-0.044980
3,-0.044980,64.343712,4.014966,1.450207,24.054663
4,181.003143,241.969513,2.938619,-0.044980,2.889809
